In [1]:
# Dependecies
import random
import requests
import pandas as pd
import time
import openweathermapy.core as owm
import matplotlib.pyplot as plt
# from config import api_key
import numpy as np
import csv
import datetime
import seaborn as sns
import calendar

In [2]:
#Import CSV/API
csv_path = "gun-violence-data_01-2013_03-2018.csv"
GunVoilenceRawData_df = pd.read_csv(csv_path)

In [3]:
# Include additional columns Year and Month
GunVoilenceRawData_df["Year"] = pd.DatetimeIndex(GunVoilenceRawData_df['date']).year
GunVoilenceRawData_df["Month"] = pd.DatetimeIndex(GunVoilenceRawData_df['date']).month
GunVoilenceRawData_df['Month'] = GunVoilenceRawData_df['Month'].apply(lambda x: calendar.month_abbr[x])

In [4]:
#Data Cleanup - Replace "District of Columbia" value with "Washington"
GunVoilenceRawData_df = GunVoilenceRawData_df.replace('District of Columbia', 'Maryland')

In [5]:
# Print cleaned up date
GunVoilenceRawData_df.head()

,incident_id,date,state,city_or_county,n_killed,n_injured,congressional_district,n_guns_involved,participant_age,participant_age_group,participant_gender,Year,Month
0,461105,1/1/2013,Pennsylvania,Mckeesport,0,4,14.0,NaN,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,2013,Jan
1,460726,1/1/2013,California,Hawthorne,1,3,43.0,NaN,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,2013,Jan
2,478855,1/1/2013,Ohio,Lorain,1,3,9.0,2.0,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,2013,Jan
3,478925,1/5/2013,Colorado,Aurora,4,0,6.0,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,2013,Jan
4,478959,1/7/2013,North Carolina,Greensboro,2,2,6.0,2.0,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,2013,Jan


In [6]:
#Find valuable data using lists - Year on Year count per State
Y2YStateData = pd.DataFrame(GunVoilenceRawData_df.groupby(['state','Year']).size())
Y2YStateData= Y2YStateData.reset_index()
Y2YStateData.columns = ['state', 'Year','ShootingCount']
Y2YStateData.count()

state            286
Year             286
ShootingCount    286
dtype: int64

In [7]:
# Melody to plot

In [8]:
# Number of killed by state
StateData = pd.DataFrame(Y2YStateData.groupby(['state'])['ShootingCount'].sum())
StateData= StateData.reset_index()
StateData

,state,ShootingCount
0,Alabama,5471
1,Alaska,1349
2,Arizona,2328
3,Arkansas,2842
4,California,16306
5,Colorado,3201
6,Connecticut,3067
7,Delaware,1685
8,Florida,15029
9,Georgia,8925


In [9]:
# Melody to plot

In [10]:
# Number of killed vs injured data
USAKilled = pd.DataFrame(GunVoilenceRawData_df.groupby(['state'])['n_killed'].sum())
USAKilled = USAKilled.reset_index()
USAInjured = pd.DataFrame(GunVoilenceRawData_df.groupby(['state'])['n_injured'].sum())
USAInjured= USAInjured.reset_index()
USAInjuredvsKilled = pd.merge(USAKilled,USAInjured, on=['state'])
USAInjuredvsKilled["Killed%"] = USAInjuredvsKilled["n_killed"]/(USAInjuredvsKilled["n_killed"]+USAInjuredvsKilled["n_injured"])
USAInjuredvsKilled["Injured%"] = USAInjuredvsKilled["n_injured"]/(USAInjuredvsKilled["n_killed"]+USAInjuredvsKilled["n_injured"])
USAInjuredvsKilled.head()

,state,n_killed,n_injured,Killed%,Injured%
0,Alabama,1880,2998,0.385404,0.614596
1,Alaska,267,325,0.451014,0.548986
2,Arizona,1094,1096,0.499543,0.500457
3,Arkansas,773,1347,0.364623,0.635377
4,California,5562,7644,0.421172,0.578828


In [11]:
# Melody to plot

In [12]:
# Texas and California - Number of killed vs injured data
TexasCalifornia = USAInjuredvsKilled.loc[USAInjuredvsKilled['state'].isin(['Texas','California'])]
TexasCalifornia

,state,n_killed,n_injured,Killed%,Injured%
4,California,5562,7644,0.421172,0.578828
42,Texas,5046,6106,0.452475,0.547525


In [13]:
#People killed by month
MonthWiseKilled = pd.DataFrame(GunVoilenceRawData_df.groupby(['Year','Month'])['n_killed'].sum())
MonthWiseKilled= MonthWiseKilled.reset_index()
MonthWiseKilled.head()

,Year,Month,n_killed
0,2013,Apr,26
1,2013,Aug,35
2,2013,Dec,29
3,2013,Feb,14
4,2013,Jan,23


In [14]:
# Melody to plot

In [15]:
#People killed by month
MonthWiseInjured = pd.DataFrame(GunVoilenceRawData_df.groupby(['Year','Month'])['n_injured'].sum())
MonthWiseInjured= MonthWiseInjured.reset_index()
MonthWiseInjured.head()

,Year,Month,n_injured
0,2013,Apr,52
1,2013,Aug,84
2,2013,Dec,57
3,2013,Feb,45
4,2013,Jan,44


In [16]:
#Gun violence by congressional district
DistrictWise_Data = pd.DataFrame(GunVoilenceRawData_df.groupby(['state','congressional_district']).size())
DistrictWise_Data= DistrictWise_Data.reset_index()
DistrictWise_Data.columns = ['state', 'congressional_district','ShootingCount']
DistrictWise_Data.count()

state                     446
congressional_district    446
ShootingCount             446
dtype: int64

In [17]:
# Melody to plot

In [18]:
#Loop Through Info

In [19]:
#Merge/Rename columns if any overlap with different data

In [ ]:
#Add columns to generate more specific data that could be useful for analysis

In [ ]:
#Add Bins putting data into catagories if numbers are involved

In [ ]:
# Find total/average data - use indexes

In [ ]:
#Sort Data 

In [ ]:
#Find amount of tweets/social media used to cover the topic chosen

In [ ]:
#Use API url/query to gather information to determine other data

In [ ]:
#Create Dataframe for Data showing the type of data, when it occured, how it occured, etc.

In [ ]:
#Plot Pie Graph to show percentages of Data

In [ ]:
#Plot Bar Graph/Line Graph/Scatter to show correlations between Data 

In [ ]:
#Explain trend(Education level vs. Unemployment level per zones)
#Explain trend (Education level vs. Median Age)
#Explain trend (Education level vs. Household income)
#Explain trend 

In [ ]:
#Export Graph images